In [1]:
import os
import base64
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file.")

client = OpenAI(api_key = openai_api_key)

with open("visa_information.json", "r") as file:
    visa_data = json.load(file)

In [3]:
def check_visa(passport_country_code, destination_country_code):
    if destination_country_code not in visa_data:
        return {"status": "applyOffline", "max_stay": None}

    destination_info = visa_data[destination_country_code]

    if "visaFree" in destination_info and passport_country_code in destination_info["visaFree"]:
        max_stay = destination_info["visaFree"][passport_country_code].get("maxStay")
        return {"status": "visaFree", "max_stay": max_stay}

    if "visaOnArrival" in destination_info and passport_country_code in destination_info["visaOnArrival"]:
        max_stay = destination_info["visaOnArrival"][passport_country_code].get("maxStay")
        return {"status": "visaOnArrival", "max_stay": max_stay}

    return {"status": "applyOffline", "max_stay": None}

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_visa_requirements",
            "description": "Determine visa requirements (visa type, max stay) for travel between two countries based ONLY on provided data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "passport_country_code": {
                        "type": "string",
                        "description": "The ISO 3166-1 alpha-3 code for the traveler's passport issuing country e.g. LBN for Lebanon, SGP for Singapore.",
                    },
                    "destination_country_code": {
                        "type": "string",
                        "description": "The ISO 3166-1 alpha-3 code for the country the traveler intends to visit e.g. PRT for Portugal",
                    },
                },
                "required": ["passport_country_code", "destination_country_code"],
            },
        }
    }
]

model_name = "gpt-4o-mini" 

system_prompt = """You are a travel visa assistant. Your role is ONLY to help users determine if they need a visa based on their passport and destination country.
Follow these steps:
- Ask for passport country and destination country if missing.
- DO NOT give any visa information before BOTH passport country and destination country are given.
- Users can upload an image of their passport. If they mention uploading or provide a file path, assume it's a passport image.
- If the image is unclear, doesn't show the country, or isn't a passport, inform the user and ask them to provide their passport country via text.
- Determine the ISO 3166-1 alpha-3 code for BOTH the passport country and the destination country.
- Use only the provided function (get_visa_requirements) for visa checks.
- Thank the user for providing the info and respond based on these statuses:
   - visaFree: Tell the user they don't need a visa and mention the maximum stay in days if available.
   - visaOnArrival: Tell the user they can get a visa on arrival and mention the maximum stay in days if available.
   - applyOffline: Tell the user they need to apply for a visa before traveling, stating that visa-free or visa-on-arrival options are not available.
   - unknownCountry: Tell the user you couldn't recognize one of the countries and ask them to clarify.
- Do not provide any visa information based on your own knowledge, only use the result from the function call.
- Keep responses concise, friendly, and professional.
- Mirror the language the user uses to communicate e.g. English, Arabic, Spanish"""

In [5]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [6]:
def run_conversation():
    messages = [{"role": "system", "content": system_prompt}]

    while True:
        user_input = input("You: ").strip()
        
        try:
            base64_image = encode_image(user_input)
            content = [
                {
                    "type": "text",
                    "text": "User uploaded an image of their passport."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
            messages.append({"role": "user", "content": content})

        except Exception:
            messages.append({"role": "user", "content": user_input})

        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages,
                tools=tools,
                tool_choice="auto",
            )
            response_message = response.choices[0].message

            tool_calls = response_message.tool_calls

            if tool_calls:
                messages.append(response_message)
                
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    
                    if function_name == "get_visa_requirements":
                        try:
                            function_args = json.loads(tool_call.function.arguments)
                            
                            function_response = check_visa(
                                passport_country_code=function_args.get("passport_country_code"),
                                destination_country_code=function_args.get("destination_country_code")
                            )

                            messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": json.dumps(function_response),
                            })
                        except Exception as e:
                            error_message = f"Error processing request: {str(e)}"
                            messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": json.dumps({"error": error_message}),
                            })

                second_response = client.chat.completions.create(
                    model=model_name,
                    messages=messages,
                )
                final_response_message = second_response.choices[0].message
                print(f"AI: {final_response_message.content}")
                messages.append(final_response_message)

            else:
                assistant_response = response_message.content
                print(f"AI: {assistant_response}")
                messages.append({"role": "assistant", "content": assistant_response})

        except Exception as e:
            print(f"AI: Error: {e}")

In [ ]:
run_conversation()

AI: I'm still not receiving any messages from you. If you need assistance, please try again!
